# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import csv
import datetime
import time
from citipy import citipy
import random
from scipy.stats import linregress

# Google API Key
from api_keys import g_key

## Generate Cities List

In [2]:
# Setup the ranges for latitude and longitude
latitude_range = (-90,90)
longatude_range = (-180,180)

# Create variables for data
cities = []

no_cities = 1500
for cntr in range(no_cities):
    latitude = random.uniform(latitude_range[0],latitude_range[1])
    longitude = random.uniform(longatude_range[0],longatude_range[1])
    
    # If we have chosen a latitude of 0 then we don't want to use that
    # one
    if latitude != 0:
        city = citipy.nearest_city(latitude,longitude)
        city_name = city.city_name
        city_country = city.country_code
        if not any(c['name'] == city_name and c['country'] == city_country for c in cities):
            city_dict = {'name': city_name,'country': city_country}
            cities.append(city_dict)

In [3]:
print(cities)

[{'name': 'kintampo', 'country': 'gh'}, {'name': 'clyde river', 'country': 'ca'}, {'name': 'zabol', 'country': 'ir'}, {'name': 'bluff', 'country': 'nz'}, {'name': 'saskylakh', 'country': 'ru'}, {'name': 'hermanus', 'country': 'za'}, {'name': 'bengkulu', 'country': 'id'}, {'name': 'mendeleyevsk', 'country': 'ru'}, {'name': 'ushuaia', 'country': 'ar'}, {'name': 'albany', 'country': 'au'}, {'name': 'illoqqortoormiut', 'country': 'gl'}, {'name': 'cabo san lucas', 'country': 'mx'}, {'name': 'penticton', 'country': 'ca'}, {'name': 'provideniya', 'country': 'ru'}, {'name': 'tuktoyaktuk', 'country': 'ca'}, {'name': 'samusu', 'country': 'ws'}, {'name': 'nelson bay', 'country': 'au'}, {'name': 'hilo', 'country': 'us'}, {'name': 'busselton', 'country': 'au'}, {'name': 'hurricane', 'country': 'us'}, {'name': 'samut songkhram', 'country': 'th'}, {'name': 'cairns', 'country': 'au'}, {'name': 'mataura', 'country': 'pf'}, {'name': 'port hardy', 'country': 'ca'}, {'name': 'cape town', 'country': 'za'},

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression